In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-05-13 11:34:52--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   159MB/s    in 0.9s    

2020-05-13 11:34:53 (159 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-05-13 11:35:38--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  55.6MB/s    in 0.2s    

2020-05-13 11:35:39 (55.6 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



In [0]:
import os   #to give access to file systems
import zipfile    #TO UNZIP THE DATA to tmp directory
local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [0]:
#directory with training horse picture
train_horse_dir=os.path.join('/tmp/horse-or-human/horses')

#directory with training human pictures
train_human_dir=os.path.join('/tmp/horse-or-human/humans')

#directory with validation horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [18]:
train_horse_names=os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names=os.listdir(train_human_dir)
print(train_human_names[:10])

['horse38-1.png', 'horse31-0.png', 'horse16-6.png', 'horse48-1.png', 'horse30-9.png', 'horse28-9.png', 'horse08-7.png', 'horse06-2.png', 'horse03-6.png', 'horse34-4.png']
['human11-18.png', 'human02-20.png', 'human03-15.png', 'human13-20.png', 'human15-13.png', 'human09-03.png', 'human13-27.png', 'human11-03.png', 'human15-28.png', 'human04-26.png']


In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:


train_datagen= ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(
    '/tmp/horse-or-human/', target_size=(300,300), batch_size=128, class_mode='binary')

test_datagen= ImageDataGenerator(rescale=1./255)
validation_generator=test_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human/', target_size=(300,300), batch_size=128, class_mode='binary')
  

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [0]:
model=tf.keras.models.Sequential([
                                  tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(300,300,3)),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                  tf.keras.layers.MaxPool2D(2,2),
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512,activation='relu'),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
]
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer= RMSprop(lr=0.001), metrics='acc')



In [12]:
history=model.fit_generator(train_generator, steps_per_epoch=8, epochs=15,validation_data=validation_generator, validation_steps=8,  verbose=2)



Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 - 101s - loss: 4.3828 - acc: 0.6045 - val_loss: 0.8377 - val_acc: 0.5156
Epoch 2/15
8/8 - 99s - loss: 0.6277 - acc: 0.6496 - val_loss: 0.3605 - val_acc: 0.8369
Epoch 3/15
8/8 - 90s - loss: 0.4280 - acc: 0.8031 - val_loss: 0.6602 - val_acc: 0.7891
Epoch 4/15
8/8 - 90s - loss: 0.0768 - acc: 0.9755 - val_loss: 0.7143 - val_acc: 0.8770
Epoch 5/15
8/8 - 90s - loss: 0.6461 - acc: 0.7964 - val_loss: 1.1956 - val_acc: 0.6914
Epoch 6/15
8/8 - 91s - loss: 0.2016 - acc: 0.9188 - val_loss: 1.3006 - val_acc: 0.7656
Epoch 7/15
8/8 - 91s - loss: 0.0540 - acc: 0.9867 - val_loss: 1.1590 - val_acc: 0.8164
Epoch 8/15
8/8 - 91s - loss: 0.0303 - acc: 0.9922 - val_loss: 2.0470 - val_acc: 0.7334
Epoch 9/15
8/8 - 90s - loss: 0.0210 - acc: 0.9956 - val_loss: 2.6736 - val_acc: 0.7188
Epoch 10/15
8/8 - 99s - loss: 0.0443 - acc: 0.9912 - val_loss: 1.2920 - val_acc: 0.8174
Epoch 11/15
8/8 - 100s - loss: 0.0041 - acc: 1.0000 

In [26]:
import numpy as np
from google.colab import files  #used to upload one/more images
from keras.preprocessing import image

uploaded=files.upload()     #image paths get loaded into the list called uploaded

for fn in uploaded.keys():    #used to upload images one by one
   #predicting images
   path='/content/'+fn
# load an image and prepare it to input into the model with this code (dimensions match the input dimensions that you specified ).
   img=image.load_img(path, target_size=(300,300))
   x= image.img_to_array(img)
   x= np.expand_dims(x, axis=0)
   images=np.vstack([x])
  #end of code
   classes=model.predict(images, batch_size=10)     #pass the input to the model and returns an array of classes
   print(classes[0])  #only contain one item with a value close to 0 for one class and close to 1 for the other.
   if classes[0]>0.5:
     print(fn + ' is a human')
   else:
     print(fn + ' is a horse')  
  



Saving Screenshot_2020-05-13 photo-1553284965-5dd8352ff1bd (WEBP Image, 1000 × 667 pixels).png to Screenshot_2020-05-13 photo-1553284965-5dd8352ff1bd (WEBP Image, 1000 × 667 pixels).png
[0.]
Screenshot_2020-05-13 photo-1553284965-5dd8352ff1bd (WEBP Image, 1000 × 667 pixels).pngis a horse


In [0]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();